In [1]:
import pandas as pd
import os
import collections
import h5py

In [2]:
# Relative path between notebooks and goruped output directories
relative_path = '../../../outputs'
output_vision_file = os.path.join(relative_path, 'Vision2050_maps.csv')
h5file = h5py.File(os.path.join(relative_path, 'daysim/daysim_outputs.h5'))

In [3]:
weekday_to_annual = 300
minute_to_hour = 60

In [4]:
mode_dict = {0: 'Other', 1:'Walk',2:'Bike',3:'SOV',4:'HOV2',5:'HOV3+',6:'Transit',8:'School Bus'}
annualization_factor = 300

In [5]:
def h5_to_df(h5file, table_list):
    output_dict = {}
    
    for table in table_list:
        df = pd.DataFrame()
        for field in h5file[table].keys():
            df[field] = h5file[table][field][:]
            
        output_dict[table] = df
    
    return output_dict

In [6]:
dataset = h5_to_df(h5file, table_list=['Household','Trip','Person'])

In [7]:
hh = dataset['Household']
person = dataset['Person']
trip = dataset['Trip']

In [54]:
person =pd.merge(person[['hhno','pno', 'psexpfac']],hh[['hhno','hhtaz']], on = 'hhno', suffixes=['','_x'])
trip = pd.merge(trip[['hhno', 'pno', 'opurp', 'dpurp', 'travdist', 'travtime', 'mode', 'dorp', 'dtaz', 'sov_ff_time']], person, on = ['hhno', 'pno'], suffixes=['','_x'], how = 'inner' )


In [55]:
# Delay per person (Annual Hours)
trip['delay'] = (trip['travtime']-(trip['sov_ff_time']/100.0))*weekday_to_annual/minute_to_hour
driver_trips = trip[['hhtaz', 'travdist', 'travtime', 'delay']].loc[trip['dorp']==1]
delay = pd.DataFrame(driver_trips.groupby('hhtaz').sum()['delay']/person.groupby('hhtaz').sum()['psexpfac']).reset_index()

output_df = delay
output_df.columns = ['hhtaz', 'annual hours of delay']

In [56]:
# VMT per resident per day
vmt = pd.DataFrame(driver_trips.groupby('hhtaz').sum()['travdist']/person.groupby('hhtaz').sum()['psexpfac']).reset_index()
vmt.columns = ['hhtaz', 'daily vehicle miles']
output_df = pd.merge(output_df, vmt, on = 'hhtaz')

In [57]:
# Number and Percent of People Walking, Biking, or Transiting
bike_walk_t_trips = trip[trip['mode'].isin([1,2,6])]

df = bike_walk_t_trips.groupby(['hhno','pno', 'hhtaz']).count()
df = df.reset_index()
df = df[['hhno','pno', 'hhtaz']]
df['bike_walk_t'] = 1
df['bike_walk_t'] = df['bike_walk_t'].fillna(0)


walk_bike_share = pd.DataFrame(df.groupby('hhtaz').sum()['bike_walk_t']/person.groupby('hhtaz').sum()['psexpfac']).reset_index()

walk_bike_share
walk_bike_share.columns = ['hhtaz', 'percent of people walking or biking']

output_df = pd.merge(output_df, walk_bike_share, on = 'hhtaz')


In [58]:

output_df.to_csv(output_vision_file)

In [59]:
output_df

,hhtaz,annual hours of delay,daily vehicle miles,percent of people walking or biking
0,1,23.858010,14.627642,0.301296
1,2,26.801487,12.638562,0.404762
2,3,18.531517,9.641302,0.450331
3,4,20.098752,10.335713,0.506410
4,5,24.744602,11.375731,0.455951
5,6,24.534507,12.284603,0.434921
6,7,28.375640,13.266525,0.319149
7,8,21.806341,12.214623,0.373648
8,9,22.169801,13.197108,0.353780
9,10,21.280176,12.701198,0.378613
